# Assinatura Digial RSA e de Chaum


### Grupo:

 - João Marques, a84684
 - Luis Bigas,   a76964

## Assinatura Digital RSA

In [1]:
def genRSAkey(bits):
    p = random_prime(2^(bits//2), 2^(bits//2-1), True)
    q = random_prime(2^(bits//2-1), 2^(bits//2-2),True)
    n = p*q
    m = (p-1)*(q-1)
    e = randint(2, m-1)
    while gcd(e, m) != 1:
        e = randint(2,m-1)
    d = power_mod(e, -1, m)
    PubKey = (n, e)
    Chave = (PubKey, d)
    return PubKey, d

In [2]:
PubKey,PrivKey = genRSAkey(1024)
n = PubKey[0]
e = PubKey[1]

## Assinatura

In [3]:
import hashlib
mOrig = "Nobody expects the Spanish Inquisition"

mEncoded = bytes(mOrig, 'utf-8')
HH = hashlib.sha256(mEncoded).hexdigest()
m=ZZ('0x'+HH)
m

36906822241210867014776754913837566155108892467980493944565876131413157803075

$s = m^d \mod n$

In [4]:
s = power_mod(m,PrivKey, n)
s

5318791555574514557375030941967875901342364429350909789540212099674511114554741801733410198165191658124171158479605477916200756158496047371752744622919741972329098592335922598281664588611962339934985962833200881614613639496048012564889341171789163082833799089172337609699785971498263538330666791462626114132

# Verificação

- PubKey (Chave Publica)
- mOrig  (Mensagem Original)
- s      (Assinatura da Mensagem)

In [5]:
import hashlib
mEncoded = bytes(mOrig, 'utf-8')
HH = hashlib.sha256(mEncoded).hexdigest()
ml=ZZ('0x'+HH)
ml

36906822241210867014776754913837566155108892467980493944565876131413157803075

$m = s^e \mod n$

In [6]:
m = power_mod(s,e,n)
m

36906822241210867014776754913837566155108892467980493944565876131413157803075

In [7]:
ml == m

True

In [8]:
def signRSA(PubKey,PrivKey,msg):
    mEncoded = bytes(msg, 'utf-8')
    HH = hashlib.sha256(mEncoded).hexdigest()
    m=ZZ('0x'+HH)
    s = power_mod(m,PrivKey, PubKey[0])
    return s

In [9]:
def verifyRSA(PubKey, msgOrig, msgSign):
    mEncoded = bytes(mOrig, 'utf-8')
    HH = hashlib.sha256(mEncoded).hexdigest()
    ml=ZZ('0x'+HH)
    m = power_mod(msgSign,PubKey[1],PubKey[0])

    return m==ml

In [10]:
m = "Nobody expects the Spanish Inquisition"

In [11]:
s = signRSA(PubKey,PrivKey,m)
s

5318791555574514557375030941967875901342364429350909789540212099674511114554741801733410198165191658124171158479605477916200756158496047371752744622919741972329098592335922598281664588611962339934985962833200881614613639496048012564889341171789163082833799089172337609699785971498263538330666791462626114132

In [12]:
verifyRSA(PubKey, m, s)

True

# Assinatura cega de Chaum

In [20]:
PubK, PrivK=genRSAkey(2048)
m=1234

$m' \equiv mr^e \mod n$

In [21]:
n,e=PubK
k=randint(1,n-1)
while gcd(k,n)!=1:
    k=randint(1,n-1)
print("m: ",m)
print("e: ",e)
print("n: ",n)
print("k: ",k)
print("PrivK: ",PrivK)

m:  1234
e:  992717950678421449644025572238987440302885385139560399771836611369887147919033714610803920327625150071406778781250606863147828935638490477194137540756934763998582149971993419356070588641594803804426904661466566231010411817572847615509477634533492785029566531938602998432141981409436174766764577706873547611743557578213386460803544625229610035650593236928672210433216694648596546564234125604415625671280359502684837700272608574652641458620536125695012760587368435718601067492424118832675893126165577561083992544741979904941662551049794552757867054575388282714973102356298403585168471662286162809143805662898335055997
n:  1251299074393624998303767352966597106934910970881594082482879864507344341648568838480752766718662430500143567830150847389474450508720499096813663250597633565805307845616789038224664312907409645596394590664142620587547245550663512776498547494059916612465306307927406529440362065999209616780617613945824590340772717847249933517645228770826690295781717422894493956079948

In [22]:
blind = (m * power_mod(k,e,n)) % n

$s' \equiv (m')^d \mod n$

In [23]:
sign = (power_mod(blind,PrivK,n)) % n

$s \equiv s'r^-1 \mod n$

In [24]:
s = (power_mod(k,-1,n)*sign) % n

In [25]:
print("blind: ",blind)
print("sign: ", sign)
print("Assinatura cega: ", s)

blind:  460981573394975985129448195039586209218843771501915701536009290085930981056260607335388044708874892081628182210026344527138235212783178221236799974819559014724183342662617379285449002579208678372091424460146796115688103255782861587698438397337534647167547168469914679097502730699302210421592460029589434972477064641305862785791671269690093095571053880351995375379572560495216218263669407526496185173824590906564770205259398603996631569703019509124308048694731500068069813919413804658250777037014455428282475805506713701083219344911600518975611753790734098842994682855372640036627980182849010889144710712910771864524
sign:  112528637412145584288446821497593467563945727227084305027125565073014250569158939678653761519894436449822197854294807059122735798086344442358834285395373332532930525558326657456766713468629395019042268205388839530284782585387400437680634370450586378404749766743618354575578671323171718581528516708671367374384806248698505755985618811620464114039448408782835635735683458

In [27]:
s==power_mod(m,PrivK,n)

True

In [28]:
power_mod(m,PrivK, n)

873004020159676394061599287943873948185201409468020768367954939688000606661881867209484537895875676132877348240283114480259179318964614030134764635315486162490920115152060772716598371794927575224524373494765666601348049622206134790618958796017967930783493191489922399891886202629622137242912799026143520576756469378881344560832250879787081651713280855510372235981209921472974345225167532985999780736817093568623937049301291210299477112505238930538256184165960532584573211818220495895756873302387636620581458590567885794263763564406835815074257006872573961860770469564837876728541646205379349715973130323584675760178

In [30]:
n

1251299074393624998303767352966597106934910970881594082482879864507344341648568838480752766718662430500143567830150847389474450508720499096813663250597633565805307845616789038224664312907409645596394590664142620587547245550663512776498547494059916612465306307927406529440362065999209616780617613945824590340772717847249933517645228770826690295781717422894493956079948995564946093522626903464236977916159611898685637946434830115525393865698339737506909664585871328137145028356149191418329773697712476692858722948678593077781052649123331018418354829812030880620129891539199047607431034578348225893146283876980126307187